Adaptivity
===

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

Define the geometry by 2D Netgen-OpenCascade modeling: Define rectangles and polygones, and glue them together to one shape object. OCC maintains the full geometry topology of vertices, edges, faces and solids.

In [ ]:
def MakeGeometry():
    base = Rectangle(1, 0.6).Face()
    chip = MoveTo(0.5,0.15).Line(0.15,0.15).Line(-0.15,0.15).Line(-0.15,-0.15).Close().Face()
    top = MoveTo(0.2,0.6).Rectangle(0.6,0.2).Face()
    base -= chip

    base.faces.name="base"
    chip.faces.name="chip"
    chip.faces.col=(1,0,0)
    top.faces.name="top"
    geo = Glue([base,chip,top])
    geo.edges.name="default"
    geo.edges.Min(Y).name="bot"
    Draw(geo)
    return geo

geo = MakeGeometry()

Piece-wise constant coefficients in sub-domains:

In [ ]:
mesh = geo.GenerateMesh(maxh=0.2, dim=2)
print (mesh.GetMaterials())

In [ ]:
fes = H1(mesh, order=3, dirichlet="bot", autoupdate=True)
u, v = fes.TnT()

lam = mesh.MaterialCF( { "base" : 1, "chip" : 1000, "top" : 20 } )
a = BilinearForm(lam*grad(u)*grad(v)*dx)

# heat-source in inner subdomain
f = LinearForm(1*v*dx(definedon="chip"))

c = preconditioners.MultiGrid(a, inverse="sparsecholesky")

gfu = GridFunction(fes) 

Assemble and solve problem:

In [ ]:
def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    
SolveBVP()
Draw (gfu, mesh);

Gradient recovery error estimator: Interpolate finite element flux 

$$
q_h := I_h (\lambda \nabla u_h)
$$

and take difference as element error indicator:

$$
\eta_T := \tfrac{1}{\lambda} \| q_h - \lambda \nabla u_h \|_{L_2(T)}^2
$$

In [ ]:
l = []    # l = list of estimated total error
space_flux = HDiv(mesh, order=2, autoupdate=True)
gf_flux = GridFunction(space_flux, "flux", autoupdate=True)

def CalcError():
    
    flux = lam * grad(gfu)   # the FEM-flux      
    gf_flux.Set(flux)        # interpolate into H(div)
    
    # compute estimator:
    err = 1/lam*(flux-gf_flux)*(flux-gf_flux)
    eta2 = Integrate(err, mesh, VOL, element_wise=True)
    l.append ((fes.ndof, sqrt(sum(eta2))))
    print("ndof =", fes.ndof, " toterr =", sqrt(sum(eta2)))
    
    # mark for refinement:
    maxerr = max(eta2) 
    # marking with Python loop:
    # for el in mesh.Elements():
    #    mesh.SetRefinementFlag(el, eta2[el.nr] > 0.25*maxerr)
    
    # marking using numpy vectorization:
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta2.NumPy() > 0.25*maxerr
  
CalcError()

Adaptive loop:

In [ ]:
level = 0
while fes.ndof < 50000:  
    mesh.Refine()
    SolveBVP()
    CalcError()
    level = level+1
    if level%5 == 0:
        Draw (gfu)

In [ ]:
Draw (gfu);

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")
plt.ylabel("H1 error-estimate")
ndof,err = zip(*l)
plt.plot(ndof,err, "-*")

plt.ion()
plt.show();